In [1]:
import pandas as pd
import numpy as np

from implicit.als import AlternatingLeastSquares

from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools.models import PopularModel, RandomModel, ImplicitALSWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
from pathlib import Path
import typing as tp
from tqdm import tqdm

from lightfm import LightFM

from implicit.bpr import BayesianPersonalizedRanking

from implicit.lmf import LogisticMatrixFactorization
import pickle

from sklearn.linear_model import LinearRegression

# Load data

In [2]:
%%time
# !wget https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip -O ../data/data_original.zip
# !uzip ../data/data_original.zip

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 10 µs


In [3]:
interactions = pd.read_csv('../data/data_original/interactions.csv')
users = pd.read_csv('../data/data_original/users.csv')
items = pd.read_csv('../data/data_original/items.csv')

In [4]:
Columns.Datetime = 'last_watch_dt'
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')
max_date = interactions[Columns.Datetime].max()
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (4985269, 6)
test: (490982, 6)


In [5]:
train.drop(train.query("total_dur < 300").index, inplace=True)
# отфильтруем холодных пользователей из теста
cold_users = set(test[Columns.User]) - set(train[Columns.User])
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

# Prepare features

## User features

In [6]:
users.fillna('Unknown', inplace=True)   
users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()
users

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
5,1037719,age_45_54,income_60_90,М,0
...,...,...,...,...,...
840184,529394,age_25_34,income_40_60,Ж,0
840186,80113,age_25_34,income_40_60,Ж,0
840188,312839,age_65_inf,income_60_90,Ж,0
840189,191349,age_45_54,income_40_60,М,1


In [7]:
user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
0,973171,М,sex
1,962099,М,sex
3,721985,Ж,sex
4,704055,Ж,sex
5,1037719,М,sex


## Item features

In [8]:
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()
items.head()

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


### Genre

In [9]:
# Explode genres to flatten table
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


### Content

In [10]:
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"
content_feature

,id,value,feature
0,10711,film,content_type
1,2508,film,content_type
2,10716,film,content_type
3,7868,film,content_type
4,16268,film,content_type
...,...,...,...
15958,6443,series,content_type
15959,2367,series,content_type
15960,10632,series,content_type
15961,4538,series,content_type


### Concat

In [11]:
item_features = pd.concat((genre_feature, content_feature))
item_features

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre
...,...,...,...
15958,6443,series,content_type
15959,2367,series,content_type
15960,10632,series,content_type
15961,4538,series,content_type


In [12]:
K_RECOS = 10
RANDOM_STATE = 42
NUM_THREADS = 4
N_FACTORS=32

In [13]:
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

# Optuna

In [14]:
import optuna

## Train ALS

Подберем лучшние гиперпараметры

In [22]:
def objective(trial):
    param = {
        "REGULARIZATION": trial.suggest_float("REGULARIZATION", 0, 1),
        "ALPHA": trial.suggest_int("ALPHA", 0, 20),
        "ITERATIONS": trial.suggest_int("ITERATIONS", 0, 10),
    }

    model = ImplicitALSWrapperModel(
        model=AlternatingLeastSquares(
            factors=N_FACTORS,
            regularization=param["REGULARIZATION"],
            alpha=param["ALPHA"],
            random_state=RANDOM_STATE,
            num_threads=NUM_THREADS,
            iterations=param["ITERATIONS"]
        ),
        fit_features_together=True
    )
    model.fit(dataset)
    recos = model.recommend(
        users=TEST_USERS,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )
    map = MAP(10)
    map = map.calc(reco=recos, interactions=test)

    return map


TEST_USERS = test[Columns.User].unique()
study_ALS = optuna.create_study(direction="maximize")
# study_ALS.optimize(objective, n_trials=30) # Ячейка выполняется 90 мин

[I 2023-12-05 20:11:11,766] A new study created in memory with name: no-name-126eb316-abd8-4d2e-9ef0-2d873b9e1874


[I 2023-12-05 20:12:29,600] Trial 0 finished with value: 0.07232951652668233 and parameters: {'REGULARIZATION': 0.7747508881804325, 'ALPHA': 3, 'ITERATIONS': 2}. Best is trial 0 with value: 0.07232951652668233.
/root/RecSysITMO/.venv/lib/python3.10/site-packages/rectools/dataset/features.py:420: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")
[I 2023-12-05 20:14:07,283] Trial 1 finished with value: 0.07450171380550459 and parameters: {'REGULARIZATION': 0.27385225864268625, 'ALPHA': 5, 'ITERATIONS': 3}. Best is trial 1 with value: 0.07450171380550459.
/root/RecSysITMO/.venv/lib/python3.10/site-packages/rectools/dataset/features.py:420: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")
[I 2023-12-05 20:16:30,959] Trial 2 finished with value: 0.07417678781225508 and para

In [23]:
# [I 2023-12-05 20:12:29,600] Trial 0 finished with value: 0.07232951652668233 and parameters: {'REGULARIZATION': 0.7747508881804325, 'ALPHA': 3, 'ITERATIONS': 2}. Best is trial 0 with value: 0.07232951652668233.
# [I 2023-12-05 20:14:07,283] Trial 1 finished with value: 0.07450171380550459 and parameters: {'REGULARIZATION': 0.27385225864268625, 'ALPHA': 5, 'ITERATIONS': 3}. Best is trial 1 with value: 0.07450171380550459.
# [I 2023-12-05 20:16:30,959] Trial 2 finished with value: 0.07417678781225508 and parameters: {'REGULARIZATION': 0.1218799742272143, 'ALPHA': 12, 'ITERATIONS': 5}. Best is trial 1 with value: 0.07450171380550459.
# [I 2023-12-05 20:18:30,499] Trial 3 finished with value: 0.071172368177507 and parameters: {'REGULARIZATION': 0.33114530823423827, 'ALPHA': 20, 'ITERATIONS': 4}. Best is trial 1 with value: 0.07450171380550459.
# [I 2023-12-05 20:22:10,802] Trial 4 finished with value: 0.07557063122005268 and parameters: {'REGULARIZATION': 0.5811893221679991, 'ALPHA': 15, 'ITERATIONS': 8}. Best is trial 4 with value: 0.07557063122005268.
# [I 2023-12-05 20:25:04,059] Trial 5 finished with value: 0.0764993357453616 and parameters: {'REGULARIZATION': 0.7486367707598471, 'ALPHA': 2, 'ITERATIONS': 6}. Best is trial 5 with value: 0.0764993357453616.
# [I 2023-12-05 20:28:20,378] Trial 6 finished with value: 0.07489141595493187 and parameters: {'REGULARIZATION': 0.9445544381507622, 'ALPHA': 19, 'ITERATIONS': 7}. Best is trial 5 with value: 0.0764993357453616.
# [I 2023-12-05 20:29:11,638] Trial 7 finished with value: 0.061778581216259 and parameters: {'REGULARIZATION': 0.7919133596398202, 'ALPHA': 8, 'ITERATIONS': 1}. Best is trial 5 with value: 0.0764993357453616.
# [I 2023-12-05 20:30:29,151] Trial 8 finished with value: 0.07290412758929592 and parameters: {'REGULARIZATION': 0.4513339860022828, 'ALPHA': 20, 'ITERATIONS': 2}. Best is trial 5 with value: 0.0764993357453616.
# [I 2023-12-05 20:32:31,200] Trial 9 finished with value: 0.07558513712608121 and parameters: {'REGULARIZATION': 0.38438249990265494, 'ALPHA': 9, 'ITERATIONS': 4}. Best is trial 5 with value: 0.0764993357453616.
# [I 2023-12-05 20:36:57,333] Trial 10 finished with value: 0.00013198418514526932 and parameters: {'REGULARIZATION': 0.6227551287302401, 'ALPHA': 0, 'ITERATIONS': 10}. Best is trial 5 with value: 0.0764993357453616.
# [I 2023-12-05 20:40:14,492] Trial 11 finished with value: 0.07522465556622855 and parameters: {'REGULARIZATION': 0.011200627547079067, 'ALPHA': 8, 'ITERATIONS': 7}. Best is trial 5 with value: 0.0764993357453616.
# [I 2023-12-05 20:43:01,440] Trial 12 finished with value: 9.220947228147791e-05 and parameters: {'REGULARIZATION': 0.4636337910549122, 'ALPHA': 0, 'ITERATIONS': 6}. Best is trial 5 with value: 0.0764993357453616.
# [I 2023-12-05 20:45:31,267] Trial 13 finished with value: 0.0742010112714058 and parameters: {'REGULARIZATION': 0.6086995027284758, 'ALPHA': 12, 'ITERATIONS': 5}. Best is trial 5 with value: 0.0764993357453616.
# [I 2023-12-05 20:45:59,535] Trial 14 finished with value: 7.407277222078664e-05 and parameters: {'REGULARIZATION': 0.3558601789136762, 'ALPHA': 4, 'ITERATIONS': 0}. Best is trial 5 with value: 0.0764993357453616.
# [I 2023-12-05 20:49:55,909] Trial 15 finished with value: 0.07591713907332522 and parameters: {'REGULARIZATION': 0.7739419426090494, 'ALPHA': 7, 'ITERATIONS': 9}. Best is trial 5 with value: 0.0764993357453616.
# [I 2023-12-05 20:54:24,898] Trial 16 finished with value: 0.07629927132427464 and parameters: {'REGULARIZATION': 0.7473821669602931, 'ALPHA': 6, 'ITERATIONS': 10}. Best is trial 5 with value: 0.0764993357453616.
# [I 2023-12-05 20:58:49,659] Trial 17 finished with value: 0.07580085729410407 and parameters: {'REGULARIZATION': 0.9793415192683359, 'ALPHA': 2, 'ITERATIONS': 10}. Best is trial 5 with value: 0.0764993357453616.
# [I 2023-12-05 21:02:22,810] Trial 18 finished with value: 0.07613472171214515 and parameters: {'REGULARIZATION': 0.6958720249278483, 'ALPHA': 5, 'ITERATIONS': 8}. Best is trial 5 with value: 0.0764993357453616.
# [I 2023-12-05 21:06:20,005] Trial 19 finished with value: 0.07750368209199175 and parameters: {'REGULARIZATION': 0.8739786271750147, 'ALPHA': 11, 'ITERATIONS': 9}. Best is trial 19 with value: 0.07750368209199175.
# [I 2023-12-05 21:09:56,576] Trial 20 finished with value: 0.07657434621791384 and parameters: {'REGULARIZATION': 0.8688756300046061, 'ALPHA': 11, 'ITERATIONS': 8}. Best is trial 19 with value: 0.07750368209199175.
# [I 2023-12-05 21:13:31,051] Trial 21 finished with value: 0.07640228106866133 and parameters: {'REGULARIZATION': 0.8930358251181556, 'ALPHA': 11, 'ITERATIONS': 8}. Best is trial 19 with value: 0.07750368209199175.
# [I 2023-12-05 21:16:38,373] Trial 22 finished with value: 0.07564429497736852 and parameters: {'REGULARIZATION': 0.8791982023716483, 'ALPHA': 15, 'ITERATIONS': 7}. Best is trial 19 with value: 0.07750368209199175.
# [I 2023-12-05 21:20:38,659] Trial 23 finished with value: 0.07599474225090147 and parameters: {'REGULARIZATION': 0.8511085977867667, 'ALPHA': 14, 'ITERATIONS': 9}. Best is trial 19 with value: 0.07750368209199175.
# [I 2023-12-05 21:23:30,737] Trial 24 finished with value: 0.07509167977275948 and parameters: {'REGULARIZATION': 0.9894216826780853, 'ALPHA': 10, 'ITERATIONS': 6}. Best is trial 19 with value: 0.07750368209199175.
# [I 2023-12-05 21:27:27,414] Trial 25 finished with value: 0.0762617670153505 and parameters: {'REGULARIZATION': 0.8735218768618357, 'ALPHA': 13, 'ITERATIONS': 9}. Best is trial 19 with value: 0.07750368209199175.
# [I 2023-12-05 21:30:27,078] Trial 26 finished with value: 0.07327578086412936 and parameters: {'REGULARIZATION': 0.6890860165134574, 'ALPHA': 16, 'ITERATIONS': 6}. Best is trial 19 with value: 0.07750368209199175.
# [I 2023-12-05 21:34:10,790] Trial 27 finished with value: 0.07628838566885444 and parameters: {'REGULARIZATION': 0.818730359938257, 'ALPHA': 17, 'ITERATIONS': 8}. Best is trial 19 with value: 0.07750368209199175.
# [I 2023-12-05 21:37:23,994] Trial 28 finished with value: 0.07448342186068439 and parameters: {'REGULARIZATION': 0.9067648921781133, 'ALPHA': 11, 'ITERATIONS': 7}. Best is trial 19 with value: 0.07750368209199175.
# [I 2023-12-05 21:41:21,018] Trial 29 finished with value: 0.07562743644294441 and parameters: {'REGULARIZATION': 0.8091938826898446, 'ALPHA': 2, 'ITERATIONS': 9}. Best is trial 19 with value: 0.07750368209199175.


Вывод: Лучшие значение MAP@10 (0.07750368209199175) было достигнуто на 19 trial, с параметрами: {'REGULARIZATION': 0.8739786271750147, 'ALPHA': 11, 'ITERATIONS': 9}.

## Train LightFM

Подберем гиперпараметры для LightFM

In [32]:
# N_EPOCHS = 2  # Lightfm
USER_ALPHA = 0  # Lightfm
ITEM_ALPHA = 0  # Lightfm
LEARNING_RATE = 0.05  # Lightfm
N_FACTORS = 32

def objective(trial):

    param = {
        "LEARNING_RATE": trial.suggest_float("LEARNING_RATE", 0.001, 0.5),
        "USER_ALPHA": trial.suggest_float("USER_ALPHA", 0, 0.01),
        "ITEM_ALPHA": trial.suggest_float("ITEM_ALPHA", 0, 0.01),
        "N_EPOCHS": trial.suggest_int("N_EPOCHS", 1, 50),
    }

    model = LightFMWrapperModel(
        LightFM(
            no_components=N_FACTORS,
            loss="warp",
            random_state=RANDOM_STATE,
            learning_rate=param["LEARNING_RATE"],
            user_alpha=param["USER_ALPHA"],
            item_alpha=param["ITEM_ALPHA"],
        ),
        epochs=param["N_EPOCHS"],
        # epochs=N_EPOCHS,
        num_threads=NUM_THREADS,
    )
    model.fit(dataset)
    recos = model.recommend(
        users=TEST_USERS,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )
    map = MAP(10)
    map = map.calc(reco=recos, interactions=test)

    return map


TEST_USERS = test[Columns.User].unique()
study_lightFM = optuna.create_study(direction="maximize")
# study_lightFM.optimize(objective, n_trials=100) # Ячейка выполняется ~800 мин

[I 2023-12-03 21:36:19,975] A new study created in memory with name: no-name-e490fca5-b80c-42b3-a1fd-596fbc0a0a31


[I 2023-12-03 21:40:35,026] Trial 0 finished with value: 0.07365611698577891 and parameters: {'LEARNING_RATE': 0.4984649009393329, 'USER_ALPHA': 0.007321919999674741, 'ITEM_ALPHA': 0.005649477235859905, 'N_EPOCHS': 19}. Best is trial 0 with value: 0.07365611698577891.
[I 2023-12-03 21:45:50,600] Trial 1 finished with value: 0.0731891114452507 and parameters: {'LEARNING_RATE': 0.16335992019601972, 'USER_ALPHA': 0.00939028264892124, 'ITEM_ALPHA': 0.006474766725656396, 'N_EPOCHS': 20}. Best is trial 0 with value: 0.07365611698577891.
[I 2023-12-03 21:59:09,537] Trial 2 finished with value: 0.06962478730668276 and parameters: {'LEARNING_RATE': 0.2762005494863486, 'USER_ALPHA': 0.0015471394507470748, 'ITEM_ALPHA': 0.0035071189302952376, 'N_EPOCHS': 43}. Best is trial 0 with value: 0.07365611698577891.
[I 2023-12-03 22:06:30,883] Trial 3 finished with value: 0.07675954767716515 and parameters: {'LEARNING_RATE': 0.35440482510463406, 'USER_ALPHA': 0.005544087818166458, 'ITEM_ALPHA': 0.00749932

Вывод: лучшее значение метрики было достигнуто на 3 trial (MAP@10 = 0.07675954767716515). 

Значения параметров: 

{'LEARNING_RATE': 0.35440482510463406, 'USER_ALPHA': 0.005544087818166458, 'ITEM_ALPHA': 0.007499324044528365, 'N_EPOCHS': 36}

## Some runs LightFM

In [36]:
# [FrozenTrial(number=0, state=TrialState.COMPLETE, values=[0.055769440493694325], datetime_start=datetime.datetime(2023, 12, 3, 20, 40, 57, 661207), datetime_complete=datetime.datetime(2023, 12, 3, 20, 41, 41, 775330), params={'LEARNING_RATE': 0.03591258227817066, 'USER_ALPHA': 0.0123724743885568, 'ITEM_ALPHA': 0.06835678888603378}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=0, value=None),
#  FrozenTrial(number=1, state=TrialState.COMPLETE, values=[0.03285690573926689], datetime_start=datetime.datetime(2023, 12, 3, 20, 41, 41, 778115), datetime_complete=datetime.datetime(2023, 12, 3, 20, 42, 26, 491313), params={'LEARNING_RATE': 0.021026266568278676, 'USER_ALPHA': 0.012875774555762233, 'ITEM_ALPHA': 0.09767686173910413}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=1, value=None),
#  FrozenTrial(number=2, state=TrialState.COMPLETE, values=[0.05675228894825963], datetime_start=datetime.datetime(2023, 12, 3, 20, 42, 26, 493175), datetime_complete=datetime.datetime(2023, 12, 3, 20, 43, 16, 651417), params={'LEARNING_RATE': 0.019660891996784228, 'USER_ALPHA': 0.09275264385661451, 'ITEM_ALPHA': 0.0050140265517021}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=2, value=None),
#  FrozenTrial(number=3, state=TrialState.COMPLETE, values=[0.0742788393417412], datetime_start=datetime.datetime(2023, 12, 3, 20, 43, 16, 654175), datetime_complete=datetime.datetime(2023, 12, 3, 20, 44, 5, 829270), params={'LEARNING_RATE': 0.03221961885484442, 'USER_ALPHA': 0.09101723901576111, 'ITEM_ALPHA': 0.0711320579982727}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=3, value=None),
#  FrozenTrial(number=4, state=TrialState.COMPLETE, values=[0.07752796402756566], datetime_start=datetime.datetime(2023, 12, 3, 20, 44, 5, 831174), datetime_complete=datetime.datetime(2023, 12, 3, 20, 44, 47, 211484), params={'LEARNING_RATE': 0.006153859149416982, 'USER_ALPHA': 0.07212867367736876, 'ITEM_ALPHA': 0.09255927237873281}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=4, value=None),
#  FrozenTrial(number=5, state=TrialState.COMPLETE, values=[0.024362860380549637], datetime_start=datetime.datetime(2023, 12, 3, 20, 44, 47, 213850), datetime_complete=datetime.datetime(2023, 12, 3, 20, 45, 36, 867787), params={'LEARNING_RATE': 0.016708376732691755, 'USER_ALPHA': 0.07800116566262613, 'ITEM_ALPHA': 0.006430616468700856}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=5, value=None),
#  FrozenTrial(number=6, state=TrialState.COMPLETE, values=[0.07367694732561987], datetime_start=datetime.datetime(2023, 12, 3, 20, 45, 36, 870179), datetime_complete=datetime.datetime(2023, 12, 3, 20, 46, 18, 66918), params={'LEARNING_RATE': 0.01928053000877495, 'USER_ALPHA': 0.04161753029745281, 'ITEM_ALPHA': 0.06395600120787012}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=6, value=None),
#  FrozenTrial(number=7, state=TrialState.COMPLETE, values=[0.015027200262952177], datetime_start=datetime.datetime(2023, 12, 3, 20, 46, 18, 68723), datetime_complete=datetime.datetime(2023, 12, 3, 20, 47, 3, 56173), params={'LEARNING_RATE': 0.04218795764149525, 'USER_ALPHA': 0.005273025579982715, 'ITEM_ALPHA': 0.05949037419204322}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=7, value=None),
#  FrozenTrial(number=8, state=TrialState.COMPLETE, values=[0.004049370382204926], datetime_start=datetime.datetime(2023, 12, 3, 20, 47, 3, 58385), datetime_complete=datetime.datetime(2023, 12, 3, 20, 47, 51, 808902), params={'LEARNING_RATE': 0.0035198956849776387, 'USER_ALPHA': 0.058212435374836814, 'ITEM_ALPHA': 0.06394192754129534}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=8, value=None),
#  FrozenTrial(number=9, state=TrialState.COMPLETE, values=[0.07476648860996458], datetime_start=datetime.datetime(2023, 12, 3, 20, 47, 51, 810641), datetime_complete=datetime.datetime(2023, 12, 3, 20, 48, 34, 755662), params={'LEARNING_RATE': 0.011095157740857835, 'USER_ALPHA': 0.07830097149632084, 'ITEM_ALPHA': 0.0878595689866288}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=9, value=None),
#  FrozenTrial(number=10, state=TrialState.COMPLETE, values=[0.0738612533911594], datetime_start=datetime.datetime(2023, 12, 3, 20, 48, 34, 757476), datetime_complete=datetime.datetime(2023, 12, 3, 20, 49, 17, 340746), params={'LEARNING_RATE': 0.002952875104434479, 'USER_ALPHA': 0.056755854848734036, 'ITEM_ALPHA': 0.09964885126574193}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=10, value=None),
#  FrozenTrial(number=11, state=TrialState.COMPLETE, values=[0.07475178673985458], datetime_start=datetime.datetime(2023, 12, 3, 20, 49, 17, 343188), datetime_complete=datetime.datetime(2023, 12, 3, 20, 50, 0, 625397), params={'LEARNING_RATE': 0.010058947732481992, 'USER_ALPHA': 0.07434346263926767, 'ITEM_ALPHA': 0.08715729909808538}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=11, value=None),
#  FrozenTrial(number=12, state=TrialState.COMPLETE, values=[0.0005424032407395317], datetime_start=datetime.datetime(2023, 12, 3, 20, 50, 0, 627270), datetime_complete=datetime.datetime(2023, 12, 3, 20, 50, 42, 115775), params={'LEARNING_RATE': 0.00950937415511073, 'USER_ALPHA': 0.0760259560581884, 'ITEM_ALPHA': 0.08318923142635391}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=12, value=None),
#  FrozenTrial(number=13, state=TrialState.COMPLETE, values=[0.07476133235530256], datetime_start=datetime.datetime(2023, 12, 3, 20, 50, 42, 117895), datetime_complete=datetime.datetime(2023, 12, 3, 20, 52, 36, 685552), params={'LEARNING_RATE': 0.011271581893790978, 'USER_ALPHA': 0.09344775055017848, 'ITEM_ALPHA': 0.08231437558577932}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=13, value=None),
#  FrozenTrial(number=14, state=TrialState.COMPLETE, values=[0.06630589556593135], datetime_start=datetime.datetime(2023, 12, 3, 20, 52, 36, 687320), datetime_complete=datetime.datetime(2023, 12, 3, 20, 53, 16, 931300), params={'LEARNING_RATE': 0.0012912278992476458, 'USER_ALPHA': 0.06863898879542253, 'ITEM_ALPHA': 0.04443859673825275}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=14, value=None),
#  FrozenTrial(number=15, state=TrialState.COMPLETE, values=[0.07371200973013912], datetime_start=datetime.datetime(2023, 12, 3, 20, 53, 16, 933683), datetime_complete=datetime.datetime(2023, 12, 3, 20, 54, 5, 528852), params={'LEARNING_RATE': 0.02680159487726458, 'USER_ALPHA': 0.09914727561400388, 'ITEM_ALPHA': 0.09816725495634376}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=15, value=None),
#  FrozenTrial(number=16, state=TrialState.COMPLETE, values=[0.07393911019068618], datetime_start=datetime.datetime(2023, 12, 3, 20, 54, 5, 530765), datetime_complete=datetime.datetime(2023, 12, 3, 20, 54, 48, 889118), params={'LEARNING_RATE': 0.012567789801113482, 'USER_ALPHA': 0.041851927569326236, 'ITEM_ALPHA': 0.04751029303979015}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=16, value=None),
#  FrozenTrial(number=17, state=TrialState.COMPLETE, values=[0.07417471072786133], datetime_start=datetime.datetime(2023, 12, 3, 20, 54, 48, 891304), datetime_complete=datetime.datetime(2023, 12, 3, 20, 55, 33, 384326), params={'LEARNING_RATE': 0.007105233239256514, 'USER_ALPHA': 0.0840348902890016, 'ITEM_ALPHA': 0.08093853255432634}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=17, value=None),
#  FrozenTrial(number=18, state=TrialState.COMPLETE, values=[0.07647924297428751], datetime_start=datetime.datetime(2023, 12, 3, 20, 55, 33, 386717), datetime_complete=datetime.datetime(2023, 12, 3, 20, 56, 21, 223752), params={'LEARNING_RATE': 0.049790154679159274, 'USER_ALPHA': 0.06668655997659494, 'ITEM_ALPHA': 0.08805475141204445}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=18, value=None),
#  FrozenTrial(number=19, state=TrialState.COMPLETE, values=[0.0729443818921789], datetime_start=datetime.datetime(2023, 12, 3, 20, 56, 21, 225859), datetime_complete=datetime.datetime(2023, 12, 3, 20, 57, 9, 424634), params={'LEARNING_RATE': 0.0466926467330102, 'USER_ALPHA': 0.06667025176157361, 'ITEM_ALPHA': 0.07561444538282744}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.05, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=19, value=None)]

In [35]:
# [FrozenTrial(number=0, state=TrialState.COMPLETE, values=[0.0743442928838727], datetime_start=datetime.datetime(2023, 12, 3, 21, 5, 45, 186541), datetime_complete=datetime.datetime(2023, 12, 3, 21, 6, 52, 195754), params={'LEARNING_RATE': 0.35231994752511187, 'USER_ALPHA': 0.006082754867576419, 'ITEM_ALPHA': 0.00944561137921526}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=0, value=None),
#  FrozenTrial(number=1, state=TrialState.COMPLETE, values=[0.06334699523932132], datetime_start=datetime.datetime(2023, 12, 3, 21, 6, 52, 198116), datetime_complete=datetime.datetime(2023, 12, 3, 21, 7, 52, 430513), params={'LEARNING_RATE': 0.05376359616544723, 'USER_ALPHA': 0.004062411220285637, 'ITEM_ALPHA': 0.00984850859307414}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=1, value=None),
#  FrozenTrial(number=2, state=TrialState.COMPLETE, values=[0.02942418607334209], datetime_start=datetime.datetime(2023, 12, 3, 21, 7, 52, 432518), datetime_complete=datetime.datetime(2023, 12, 3, 21, 8, 50, 912720), params={'LEARNING_RATE': 0.0966037182976569, 'USER_ALPHA': 0.005421439370909161, 'ITEM_ALPHA': 0.0008385971236838385}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=2, value=None),
#  FrozenTrial(number=3, state=TrialState.COMPLETE, values=[0.07388988418974869], datetime_start=datetime.datetime(2023, 12, 3, 21, 8, 50, 914568), datetime_complete=datetime.datetime(2023, 12, 3, 21, 10, 18, 114826), params={'LEARNING_RATE': 0.25977511101879, 'USER_ALPHA': 0.004528552111317469, 'ITEM_ALPHA': 0.008148478532813471}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=3, value=None),
#  FrozenTrial(number=4, state=TrialState.COMPLETE, values=[0.06939354303633397], datetime_start=datetime.datetime(2023, 12, 3, 21, 10, 18, 117095), datetime_complete=datetime.datetime(2023, 12, 3, 21, 11, 27, 936123), params={'LEARNING_RATE': 0.3972209916784743, 'USER_ALPHA': 0.004313599845877805, 'ITEM_ALPHA': 0.007601257436352695}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=4, value=None),
#  FrozenTrial(number=5, state=TrialState.COMPLETE, values=[0.008437688985263028], datetime_start=datetime.datetime(2023, 12, 3, 21, 11, 27, 938138), datetime_complete=datetime.datetime(2023, 12, 3, 21, 12, 46, 229062), params={'LEARNING_RATE': 0.30012564859071084, 'USER_ALPHA': 0.000866055012772704, 'ITEM_ALPHA': 0.0072424836979662465}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=5, value=None),
#  FrozenTrial(number=6, state=TrialState.COMPLETE, values=[0.07622449657499258], datetime_start=datetime.datetime(2023, 12, 3, 21, 12, 46, 230809), datetime_complete=datetime.datetime(2023, 12, 3, 21, 13, 38, 695459), params={'LEARNING_RATE': 0.2677992128776159, 'USER_ALPHA': 0.002972999795179375, 'ITEM_ALPHA': 0.0013614396158947006}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=6, value=None),
#  FrozenTrial(number=7, state=TrialState.COMPLETE, values=[0.07377494039319862], datetime_start=datetime.datetime(2023, 12, 3, 21, 13, 38, 697416), datetime_complete=datetime.datetime(2023, 12, 3, 21, 14, 31, 385089), params={'LEARNING_RATE': 0.18746420638233752, 'USER_ALPHA': 0.003450242367558014, 'ITEM_ALPHA': 0.008577410634675865}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=7, value=None),
#  FrozenTrial(number=8, state=TrialState.COMPLETE, values=[0.07189270882066653], datetime_start=datetime.datetime(2023, 12, 3, 21, 14, 31, 387836), datetime_complete=datetime.datetime(2023, 12, 3, 21, 15, 56, 304375), params={'LEARNING_RATE': 0.23771867971644425, 'USER_ALPHA': 0.005691189914664381, 'ITEM_ALPHA': 0.0047994372347828045}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=8, value=None),
#  FrozenTrial(number=9, state=TrialState.COMPLETE, values=[0.07356839433321696], datetime_start=datetime.datetime(2023, 12, 3, 21, 15, 56, 306661), datetime_complete=datetime.datetime(2023, 12, 3, 21, 17, 1, 724890), params={'LEARNING_RATE': 0.21314992776280106, 'USER_ALPHA': 0.006778749379714221, 'ITEM_ALPHA': 0.007250550025728726}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=9, value=None),
#  FrozenTrial(number=10, state=TrialState.COMPLETE, values=[2.7821807135285596e-05], datetime_start=datetime.datetime(2023, 12, 3, 21, 17, 1, 727571), datetime_complete=datetime.datetime(2023, 12, 3, 21, 18, 12, 916325), params={'LEARNING_RATE': 0.4877944416442691, 'USER_ALPHA': 0.009142700242252793, 'ITEM_ALPHA': 1.1922258116742537e-05}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=10, value=None),
#  FrozenTrial(number=11, state=TrialState.COMPLETE, values=[0.07330971212970284], datetime_start=datetime.datetime(2023, 12, 3, 21, 18, 12, 918469), datetime_complete=datetime.datetime(2023, 12, 3, 21, 19, 5, 981831), params={'LEARNING_RATE': 0.35262493894606006, 'USER_ALPHA': 0.0015274243662524727, 'ITEM_ALPHA': 0.002637258214623648}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=11, value=None),
#  FrozenTrial(number=12, state=TrialState.COMPLETE, values=[0.07330375662652801], datetime_start=datetime.datetime(2023, 12, 3, 21, 19, 5, 983922), datetime_complete=datetime.datetime(2023, 12, 3, 21, 21, 34, 484754), params={'LEARNING_RATE': 0.35748777138563614, 'USER_ALPHA': 0.0026788270950978627, 'ITEM_ALPHA': 0.004666765434010741}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=12, value=None),
#  FrozenTrial(number=13, state=TrialState.COMPLETE, values=[0.07266319900352444], datetime_start=datetime.datetime(2023, 12, 3, 21, 21, 34, 486581), datetime_complete=datetime.datetime(2023, 12, 3, 21, 22, 27, 585817), params={'LEARNING_RATE': 0.16661034001693992, 'USER_ALPHA': 0.0024120701498466473, 'ITEM_ALPHA': 0.0029206458099122284}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=13, value=None),
#  FrozenTrial(number=14, state=TrialState.COMPLETE, values=[1.4566378785739812e-05], datetime_start=datetime.datetime(2023, 12, 3, 21, 22, 27, 587747), datetime_complete=datetime.datetime(2023, 12, 3, 21, 23, 33, 596961), params={'LEARNING_RATE': 0.30026401811996223, 'USER_ALPHA': 0.0003250653240720802, 'ITEM_ALPHA': 0.006257212789267657}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=14, value=None),
#  FrozenTrial(number=15, state=TrialState.COMPLETE, values=[0.07368647500599901], datetime_start=datetime.datetime(2023, 12, 3, 21, 23, 33, 599280), datetime_complete=datetime.datetime(2023, 12, 3, 21, 25, 36, 87208), params={'LEARNING_RATE': 0.13967674152500326, 'USER_ALPHA': 0.006498971487955635, 'ITEM_ALPHA': 0.009724743984824483}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=15, value=None),
#  FrozenTrial(number=16, state=TrialState.COMPLETE, values=[0.07462603798641151], datetime_start=datetime.datetime(2023, 12, 3, 21, 25, 36, 89157), datetime_complete=datetime.datetime(2023, 12, 3, 21, 26, 37, 941295), params={'LEARNING_RATE': 0.026956659439011016, 'USER_ALPHA': 0.007552073351519933, 'ITEM_ALPHA': 0.005779595803600757}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=16, value=None),
#  FrozenTrial(number=17, state=TrialState.COMPLETE, values=[0.07455029725123058], datetime_start=datetime.datetime(2023, 12, 3, 21, 26, 37, 943229), datetime_complete=datetime.datetime(2023, 12, 3, 21, 27, 40, 841217), params={'LEARNING_RATE': 0.02910943314621831, 'USER_ALPHA': 0.007728999177043173, 'ITEM_ALPHA': 0.005567512577887827}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=17, value=None),
#  FrozenTrial(number=18, state=TrialState.COMPLETE, values=[0.07624132776590753], datetime_start=datetime.datetime(2023, 12, 3, 21, 27, 40, 843044), datetime_complete=datetime.datetime(2023, 12, 3, 21, 28, 34, 794311), params={'LEARNING_RATE': 0.10175804447915071, 'USER_ALPHA': 0.009882732341370085, 'ITEM_ALPHA': 0.0035765279038818907}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=18, value=None),
#  FrozenTrial(number=19, state=TrialState.COMPLETE, values=[0.07703793416860495], datetime_start=datetime.datetime(2023, 12, 3, 21, 28, 34, 796192), datetime_complete=datetime.datetime(2023, 12, 3, 21, 29, 32, 510871), params={'LEARNING_RATE': 0.11959510042489116, 'USER_ALPHA': 0.009599497541474227, 'ITEM_ALPHA': 0.003192548505805458}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'LEARNING_RATE': FloatDistribution(high=0.5, log=False, low=0.001, step=None), 'USER_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'ITEM_ALPHA': FloatDistribution(high=0.01, log=False, low=0.0, step=None)}, trial_id=19, value=None)]

In [34]:
# [I 2023-12-03 21:36:19,975] A new study created in memory with name: no-name-e490fca5-b80c-42b3-a1fd-596fbc0a0a31
# [I 2023-12-03 21:40:35,026] Trial 0 finished with value: 0.07365611698577891 and parameters: {'LEARNING_RATE': 0.4984649009393329, 'USER_ALPHA': 0.007321919999674741, 'ITEM_ALPHA': 0.005649477235859905, 'N_EPOCHS': 19}. Best is trial 0 with value: 0.07365611698577891.
# [I 2023-12-03 21:45:50,600] Trial 1 finished with value: 0.0731891114452507 and parameters: {'LEARNING_RATE': 0.16335992019601972, 'USER_ALPHA': 0.00939028264892124, 'ITEM_ALPHA': 0.006474766725656396, 'N_EPOCHS': 20}. Best is trial 0 with value: 0.07365611698577891.
# [I 2023-12-03 21:59:09,537] Trial 2 finished with value: 0.06962478730668276 and parameters: {'LEARNING_RATE': 0.2762005494863486, 'USER_ALPHA': 0.0015471394507470748, 'ITEM_ALPHA': 0.0035071189302952376, 'N_EPOCHS': 43}. Best is trial 0 with value: 0.07365611698577891.
# [I 2023-12-03 22:06:30,883] Trial 3 finished with value: 0.07675954767716515 and parameters: {'LEARNING_RATE': 0.35440482510463406, 'USER_ALPHA': 0.005544087818166458, 'ITEM_ALPHA': 0.007499324044528365, 'N_EPOCHS': 36}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-03 22:13:15,851] Trial 4 finished with value: 0.008039566585750222 and parameters: {'LEARNING_RATE': 0.0316951865108015, 'USER_ALPHA': 0.006972928667320034, 'ITEM_ALPHA': 8.242597023009447e-05, 'N_EPOCHS': 32}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-03 22:23:02,542] Trial 5 finished with value: 0.07504936945073884 and parameters: {'LEARNING_RATE': 0.02144984880107252, 'USER_ALPHA': 0.0045644039704720455, 'ITEM_ALPHA': 0.008680887047661033, 'N_EPOCHS': 33}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-03 22:28:35,536] Trial 6 finished with value: 0.0736985212539794 and parameters: {'LEARNING_RATE': 0.36720721650611804, 'USER_ALPHA': 0.007724994629980363, 'ITEM_ALPHA': 0.009741593428769712, 'N_EPOCHS': 27}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-03 22:44:47,869] Trial 7 finished with value: 0.07331118999251308 and parameters: {'LEARNING_RATE': 0.08895161063051545, 'USER_ALPHA': 0.00570534942471003, 'ITEM_ALPHA': 0.005086425417445798, 'N_EPOCHS': 49}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-03 22:47:09,627] Trial 8 finished with value: 0.0430684578274521 and parameters: {'LEARNING_RATE': 0.44201014238447783, 'USER_ALPHA': 0.0007422772907547215, 'ITEM_ALPHA': 0.00975457302043841, 'N_EPOCHS': 9}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-03 22:49:28,149] Trial 9 finished with value: 0.0065995423776200116 and parameters: {'LEARNING_RATE': 0.06050326891596844, 'USER_ALPHA': 0.0006007441758290622, 'ITEM_ALPHA': 0.007081931090881263, 'N_EPOCHS': 6}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-03 22:58:05,710] Trial 10 finished with value: 0.07595029442675473 and parameters: {'LEARNING_RATE': 0.320386221009032, 'USER_ALPHA': 0.0037868701740056583, 'ITEM_ALPHA': 0.007183876127449394, 'N_EPOCHS': 41}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-03 23:06:44,672] Trial 11 finished with value: 0.0755247424269306 and parameters: {'LEARNING_RATE': 0.31233022036689534, 'USER_ALPHA': 0.003869713659952202, 'ITEM_ALPHA': 0.007648697867242216, 'N_EPOCHS': 42}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-03 23:16:31,359] Trial 12 finished with value: 0.07512043490173174 and parameters: {'LEARNING_RATE': 0.21640093277272654, 'USER_ALPHA': 0.003036199301514826, 'ITEM_ALPHA': 0.008051016319648725, 'N_EPOCHS': 39}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-03 23:26:53,945] Trial 13 finished with value: 0.07468930684154855 and parameters: {'LEARNING_RATE': 0.3586847743641226, 'USER_ALPHA': 0.0030264790847658665, 'ITEM_ALPHA': 0.006589712951317772, 'N_EPOCHS': 50}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-03 23:34:29,366] Trial 14 finished with value: 0.07292491855277715 and parameters: {'LEARNING_RATE': 0.22715872783219584, 'USER_ALPHA': 0.005579473655964314, 'ITEM_ALPHA': 0.008369118045236974, 'N_EPOCHS': 35}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-03 23:40:20,074] Trial 15 finished with value: 0.07339164986715195 and parameters: {'LEARNING_RATE': 0.3796686316154064, 'USER_ALPHA': 0.004847411999738111, 'ITEM_ALPHA': 0.004061497771743651, 'N_EPOCHS': 26}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-03 23:50:17,354] Trial 16 finished with value: 0.07400580525650478 and parameters: {'LEARNING_RATE': 0.3004777698067414, 'USER_ALPHA': 0.002303569711053204, 'ITEM_ALPHA': 0.00594958887839102, 'N_EPOCHS': 45}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-03 23:57:51,863] Trial 17 finished with value: 0.04668595926610642 and parameters: {'LEARNING_RATE': 0.4283745506980694, 'USER_ALPHA': 0.003847703575066965, 'ITEM_ALPHA': 0.007375773964414807, 'N_EPOCHS': 38}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 00:01:46,155] Trial 18 finished with value: 0.07292957449325001 and parameters: {'LEARNING_RATE': 0.3055333634743188, 'USER_ALPHA': 0.006074602085353698, 'ITEM_ALPHA': 0.008942365973660032, 'N_EPOCHS': 17}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 00:08:06,060] Trial 19 finished with value: 0.0735824076461686 and parameters: {'LEARNING_RATE': 0.2456676507431429, 'USER_ALPHA': 0.004213188839485023, 'ITEM_ALPHA': 0.00746207892806294, 'N_EPOCHS': 29}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 00:08:45,940] Trial 20 finished with value: 0.03875553801225174 and parameters: {'LEARNING_RATE': 0.191394434644582, 'USER_ALPHA': 0.002426543036192268, 'ITEM_ALPHA': 0.00907091735098714, 'N_EPOCHS': 1}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 00:17:18,426] Trial 21 finished with value: 0.07380212987276905 and parameters: {'LEARNING_RATE': 0.32623443632202476, 'USER_ALPHA': 0.0038327611312756397, 'ITEM_ALPHA': 0.007876800153689478, 'N_EPOCHS': 43}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 00:25:49,903] Trial 22 finished with value: 0.0737070303832756 and parameters: {'LEARNING_RATE': 0.2758513952572032, 'USER_ALPHA': 0.003455945422446702, 'ITEM_ALPHA': 0.006889814531869776, 'N_EPOCHS': 40}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 00:35:10,136] Trial 23 finished with value: 0.07597007890165602 and parameters: {'LEARNING_RATE': 0.2673107543545931, 'USER_ALPHA': 0.004780958182365626, 'ITEM_ALPHA': 0.007889579103634916, 'N_EPOCHS': 46}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 00:44:39,599] Trial 24 finished with value: 0.0765112928645881 and parameters: {'LEARNING_RATE': 0.2652407753852967, 'USER_ALPHA': 0.005016197868936939, 'ITEM_ALPHA': 0.008146877216475563, 'N_EPOCHS': 46}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 00:54:03,747] Trial 25 finished with value: 0.07509947828671826 and parameters: {'LEARNING_RATE': 0.2573447219396848, 'USER_ALPHA': 0.004957007559344947, 'ITEM_ALPHA': 0.008412042734108413, 'N_EPOCHS': 47}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 01:03:26,851] Trial 26 finished with value: 0.07561946364745875 and parameters: {'LEARNING_RATE': 0.18907592134794815, 'USER_ALPHA': 0.006332239670241158, 'ITEM_ALPHA': 0.009223888135046956, 'N_EPOCHS': 46}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 01:10:47,736] Trial 27 finished with value: 0.07634438021838265 and parameters: {'LEARNING_RATE': 0.2621869738258264, 'USER_ALPHA': 0.0052032956061172225, 'ITEM_ALPHA': 0.009886981845387725, 'N_EPOCHS': 35}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 01:19:19,256] Trial 28 finished with value: 0.07532958975950849 and parameters: {'LEARNING_RATE': 0.15405604104133966, 'USER_ALPHA': 0.005435676121523476, 'ITEM_ALPHA': 0.009650316912330238, 'N_EPOCHS': 36}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 01:23:57,355] Trial 29 finished with value: 0.07587244894324335 and parameters: {'LEARNING_RATE': 0.40988177913090995, 'USER_ALPHA': 0.006594986227704713, 'ITEM_ALPHA': 0.009976514332918367, 'N_EPOCHS': 21}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 01:30:28,077] Trial 30 finished with value: 0.07402070919676344 and parameters: {'LEARNING_RATE': 0.46739588877505495, 'USER_ALPHA': 0.007620371896171594, 'ITEM_ALPHA': 0.00870019518850559, 'N_EPOCHS': 31}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 01:38:22,685] Trial 31 finished with value: 0.07600897249902815 and parameters: {'LEARNING_RATE': 0.27092034640056073, 'USER_ALPHA': 0.005087281141466688, 'ITEM_ALPHA': 0.008109907660627944, 'N_EPOCHS': 36}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 01:45:57,312] Trial 32 finished with value: 0.074541906165951 and parameters: {'LEARNING_RATE': 0.34949851245700725, 'USER_ALPHA': 0.005688917934624523, 'ITEM_ALPHA': 0.00915161845049044, 'N_EPOCHS': 36}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 01:51:08,948] Trial 33 finished with value: 0.07476197742035147 and parameters: {'LEARNING_RATE': 0.29078776037126775, 'USER_ALPHA': 0.00530854857683555, 'ITEM_ALPHA': 0.008229095973104509, 'N_EPOCHS': 23}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 01:58:03,071] Trial 34 finished with value: 0.0759265106406744 and parameters: {'LEARNING_RATE': 0.23391208328946836, 'USER_ALPHA': 0.006802257802031955, 'ITEM_ALPHA': 0.005959380305836785, 'N_EPOCHS': 30}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 02:05:28,239] Trial 35 finished with value: 0.07439182199267227 and parameters: {'LEARNING_RATE': 0.3345907377641195, 'USER_ALPHA': 0.004691164335990471, 'ITEM_ALPHA': 0.006475289099167919, 'N_EPOCHS': 34}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 02:09:20,253] Trial 36 finished with value: 0.07349773553709216 and parameters: {'LEARNING_RATE': 0.27712090475891393, 'USER_ALPHA': 0.006070238303830437, 'ITEM_ALPHA': 0.009258589911537156, 'N_EPOCHS': 16}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 02:17:06,298] Trial 37 finished with value: 0.07417828990056798 and parameters: {'LEARNING_RATE': 0.3741887666227378, 'USER_ALPHA': 0.004338324186088173, 'ITEM_ALPHA': 0.00864721368719381, 'N_EPOCHS': 37}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 02:23:08,408] Trial 38 finished with value: 0.07593344202273641 and parameters: {'LEARNING_RATE': 0.254472886585185, 'USER_ALPHA': 0.008092450141412832, 'ITEM_ALPHA': 0.008020241541650259, 'N_EPOCHS': 28}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 02:30:28,624] Trial 39 finished with value: 0.07458244246399084 and parameters: {'LEARNING_RATE': 0.2961448529611114, 'USER_ALPHA': 0.005157397250853724, 'ITEM_ALPHA': 0.009649296555858798, 'N_EPOCHS': 34}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 02:35:40,902] Trial 40 finished with value: 0.07578348191947801 and parameters: {'LEARNING_RATE': 0.38772679738274807, 'USER_ALPHA': 0.007055109798730967, 'ITEM_ALPHA': 0.009986899837381004, 'N_EPOCHS': 24}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 02:45:17,343] Trial 41 finished with value: 0.07314618268640548 and parameters: {'LEARNING_RATE': 0.2777305182011232, 'USER_ALPHA': 0.004558399686429885, 'ITEM_ALPHA': 0.007753571736300191, 'N_EPOCHS': 45}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 02:55:20,961] Trial 42 finished with value: 0.07604508166746701 and parameters: {'LEARNING_RATE': 0.34021782724393124, 'USER_ALPHA': 0.005100017571197151, 'ITEM_ALPHA': 0.00731070434439384, 'N_EPOCHS': 49}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 03:05:14,904] Trial 43 finished with value: 0.05697512670375448 and parameters: {'LEARNING_RATE': 0.3591383911896883, 'USER_ALPHA': 0.006009493616124182, 'ITEM_ALPHA': 0.0068639864644147435, 'N_EPOCHS': 49}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 03:13:55,938] Trial 44 finished with value: 0.07505321243867112 and parameters: {'LEARNING_RATE': 0.3420245207350395, 'USER_ALPHA': 0.005224099255648546, 'ITEM_ALPHA': 0.007328123069202145, 'N_EPOCHS': 42}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 03:22:45,185] Trial 45 finished with value: 0.05841071528679995 and parameters: {'LEARNING_RATE': 0.3200804443648879, 'USER_ALPHA': 0.005674511986147263, 'ITEM_ALPHA': 0.008532338400206091, 'N_EPOCHS': 43}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 03:31:02,651] Trial 46 finished with value: 0.07341155124325231 and parameters: {'LEARNING_RATE': 0.3376320855956184, 'USER_ALPHA': 0.004315308358537132, 'ITEM_ALPHA': 0.006974500379948124, 'N_EPOCHS': 39}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 03:40:51,873] Trial 47 finished with value: 0.07587751709436426 and parameters: {'LEARNING_RATE': 0.3137533056291631, 'USER_ALPHA': 0.004988509845427411, 'ITEM_ALPHA': 0.006162600381230054, 'N_EPOCHS': 48}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 03:51:39,234] Trial 48 finished with value: 0.0658618984404859 and parameters: {'LEARNING_RATE': 0.24467973998696915, 'USER_ALPHA': 0.006368965049931862, 'ITEM_ALPHA': 0.005253970882567315, 'N_EPOCHS': 50}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 03:58:29,534] Trial 49 finished with value: 0.07444232458499941 and parameters: {'LEARNING_RATE': 0.29201605646799267, 'USER_ALPHA': 0.005544557489242008, 'ITEM_ALPHA': 0.007535802410696219, 'N_EPOCHS': 32}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 04:07:25,993] Trial 50 finished with value: 0.0746691636216502 and parameters: {'LEARNING_RATE': 0.21492884599480494, 'USER_ALPHA': 0.0041869996386903535, 'ITEM_ALPHA': 0.008741236578073358, 'N_EPOCHS': 40}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 04:16:57,345] Trial 51 finished with value: 0.07163179938760666 and parameters: {'LEARNING_RATE': 0.328140720073826, 'USER_ALPHA': 0.0047804727222403596, 'ITEM_ALPHA': 0.007694009755213519, 'N_EPOCHS': 45}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 04:27:04,832] Trial 52 finished with value: 0.07552314117895778 and parameters: {'LEARNING_RATE': 0.2698386931108351, 'USER_ALPHA': 0.0051902092314499056, 'ITEM_ALPHA': 0.00808860927293195, 'N_EPOCHS': 47}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 04:36:33,750] Trial 53 finished with value: 0.07331879011383183 and parameters: {'LEARNING_RATE': 0.2634072078316416, 'USER_ALPHA': 0.004634130360615614, 'ITEM_ALPHA': 0.008320673713556806, 'N_EPOCHS': 44}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 04:44:42,840] Trial 54 finished with value: 0.07477575235760268 and parameters: {'LEARNING_RATE': 0.3120397010655753, 'USER_ALPHA': 0.005935804545440723, 'ITEM_ALPHA': 0.006706242286488844, 'N_EPOCHS': 38}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 04:53:30,251] Trial 55 finished with value: 0.07328250788592391 and parameters: {'LEARNING_RATE': 0.3561502462632463, 'USER_ALPHA': 0.004936481789092479, 'ITEM_ALPHA': 0.007388761873671581, 'N_EPOCHS': 42}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 05:03:58,390] Trial 56 finished with value: 0.07274555755881779 and parameters: {'LEARNING_RATE': 0.28219802112448766, 'USER_ALPHA': 0.005488353296686706, 'ITEM_ALPHA': 0.00798036499286801, 'N_EPOCHS': 50}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 05:14:07,312] Trial 57 finished with value: 0.05951030218334483 and parameters: {'LEARNING_RATE': 0.29686867335171574, 'USER_ALPHA': 0.0035693535632293164, 'ITEM_ALPHA': 0.007136595989504603, 'N_EPOCHS': 48}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 05:17:58,097] Trial 58 finished with value: 0.07345426144264552 and parameters: {'LEARNING_RATE': 0.25775682633319397, 'USER_ALPHA': 0.003963806191242349, 'ITEM_ALPHA': 0.009002643892684249, 'N_EPOCHS': 13}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 05:27:02,996] Trial 59 finished with value: 0.07464199446840543 and parameters: {'LEARNING_RATE': 0.22326166053010987, 'USER_ALPHA': 0.004492683256010974, 'ITEM_ALPHA': 0.009408678497401004, 'N_EPOCHS': 41}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 05:36:12,918] Trial 60 finished with value: 0.07597263479213609 and parameters: {'LEARNING_RATE': 0.3928745206128126, 'USER_ALPHA': 0.0049971356236026025, 'ITEM_ALPHA': 0.006337791051140731, 'N_EPOCHS': 46}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 05:45:42,724] Trial 61 finished with value: 0.06239199301746809 and parameters: {'LEARNING_RATE': 0.38872534143574344, 'USER_ALPHA': 0.005113062449469982, 'ITEM_ALPHA': 0.007916207794875797, 'N_EPOCHS': 46}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 05:54:42,401] Trial 62 finished with value: 0.05660128286442055 and parameters: {'LEARNING_RATE': 0.36193934731517863, 'USER_ALPHA': 0.005827037982005694, 'ITEM_ALPHA': 0.00638534510400859, 'N_EPOCHS': 44}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 06:01:37,713] Trial 63 finished with value: 0.07443377282100418 and parameters: {'LEARNING_RATE': 0.3051066781863926, 'USER_ALPHA': 0.004770606272756701, 'ITEM_ALPHA': 0.007075432448047878, 'N_EPOCHS': 32}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 06:10:56,419] Trial 64 finished with value: 0.07301461526100489 and parameters: {'LEARNING_RATE': 0.40120516882820323, 'USER_ALPHA': 0.005402542268541715, 'ITEM_ALPHA': 0.00751769129210637, 'N_EPOCHS': 47}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 06:19:03,119] Trial 65 finished with value: 0.07601219902858389 and parameters: {'LEARNING_RATE': 0.3451273162523871, 'USER_ALPHA': 0.004197393746116301, 'ITEM_ALPHA': 0.008453579348207717, 'N_EPOCHS': 37}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 06:26:36,766] Trial 66 finished with value: 0.07235696408639081 and parameters: {'LEARNING_RATE': 0.3413801302380424, 'USER_ALPHA': 0.0040009767944355805, 'ITEM_ALPHA': 0.008349284498475648, 'N_EPOCHS': 35}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 06:34:06,107] Trial 67 finished with value: 0.07555261472939208 and parameters: {'LEARNING_RATE': 0.3704015267588281, 'USER_ALPHA': 0.004377999053526938, 'ITEM_ALPHA': 0.008652097317901612, 'N_EPOCHS': 37}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 06:39:45,764] Trial 68 finished with value: 0.07250485050515343 and parameters: {'LEARNING_RATE': 0.4214155966283768, 'USER_ALPHA': 0.005743995107219248, 'ITEM_ALPHA': 0.008943430445339969, 'N_EPOCHS': 27}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 06:48:19,863] Trial 69 finished with value: 0.07594562807441031 and parameters: {'LEARNING_RATE': 0.3230893541475648, 'USER_ALPHA': 0.003585337553734396, 'ITEM_ALPHA': 0.00667427331782184, 'N_EPOCHS': 40}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 06:54:40,904] Trial 70 finished with value: 0.07538916954240131 and parameters: {'LEARNING_RATE': 0.3810414085069944, 'USER_ALPHA': 0.0050686114287073975, 'ITEM_ALPHA': 0.00767544147376784, 'N_EPOCHS': 30}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 07:01:52,299] Trial 71 finished with value: 0.0748458592384253 and parameters: {'LEARNING_RATE': 0.3446319461411016, 'USER_ALPHA': 0.004732500878944599, 'ITEM_ALPHA': 0.008245379131516531, 'N_EPOCHS': 34}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 07:09:58,846] Trial 72 finished with value: 0.07618213770195897 and parameters: {'LEARNING_RATE': 0.270195082979672, 'USER_ALPHA': 0.0053642964485823815, 'ITEM_ALPHA': 0.0072208877026788505, 'N_EPOCHS': 38}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 07:18:11,902] Trial 73 finished with value: 0.07379160813773022 and parameters: {'LEARNING_RATE': 0.2926964569536378, 'USER_ALPHA': 0.005356857327520273, 'ITEM_ALPHA': 0.007168576344763117, 'N_EPOCHS': 38}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 07:26:22,651] Trial 74 finished with value: 0.07336852201203321 and parameters: {'LEARNING_RATE': 0.24065032081587528, 'USER_ALPHA': 0.004248205405231088, 'ITEM_ALPHA': 0.0077231086102849495, 'N_EPOCHS': 36}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 07:35:11,534] Trial 75 finished with value: 0.0736652326116871 and parameters: {'LEARNING_RATE': 0.3249537592595546, 'USER_ALPHA': 0.006151032714001715, 'ITEM_ALPHA': 0.007237755227206566, 'N_EPOCHS': 43}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 07:42:07,516] Trial 76 finished with value: 0.07575074643493111 and parameters: {'LEARNING_RATE': 0.283833307452912, 'USER_ALPHA': 0.00574507193994113, 'ITEM_ALPHA': 0.009387916907261842, 'N_EPOCHS': 33}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 07:50:31,571] Trial 77 finished with value: 0.05230157922999703 and parameters: {'LEARNING_RATE': 0.3070437279073346, 'USER_ALPHA': 0.005313065427705256, 'ITEM_ALPHA': 0.00813531373198288, 'N_EPOCHS': 41}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 07:58:21,013] Trial 78 finished with value: 0.060874858523315346 and parameters: {'LEARNING_RATE': 0.25942417237815285, 'USER_ALPHA': 0.00455704155890331, 'ITEM_ALPHA': 0.006886172947966568, 'N_EPOCHS': 37}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 08:05:45,957] Trial 79 finished with value: 0.05648709302055611 and parameters: {'LEARNING_RATE': 0.3327164874132598, 'USER_ALPHA': 0.0049379055657646186, 'ITEM_ALPHA': 0.00882262709795233, 'N_EPOCHS': 35}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 08:15:31,633] Trial 80 finished with value: 0.07387107134773675 and parameters: {'LEARNING_RATE': 0.35525265522239724, 'USER_ALPHA': 0.0063360734453119065, 'ITEM_ALPHA': 0.008489852037041796, 'N_EPOCHS': 48}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 08:25:14,752] Trial 81 finished with value: 0.07552424918839216 and parameters: {'LEARNING_RATE': 0.2663225806573781, 'USER_ALPHA': 0.005537830093744759, 'ITEM_ALPHA': 0.007926158265756407, 'N_EPOCHS': 46}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 08:33:42,826] Trial 82 finished with value: 0.07408637732032869 and parameters: {'LEARNING_RATE': 0.23627519100682018, 'USER_ALPHA': 0.005050882752956629, 'ITEM_ALPHA': 0.007444083843391186, 'N_EPOCHS': 39}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 08:43:23,143] Trial 83 finished with value: 0.07376659800389881 and parameters: {'LEARNING_RATE': 0.27271454689924507, 'USER_ALPHA': 0.004601182272912147, 'ITEM_ALPHA': 0.00786530384579673, 'N_EPOCHS': 45}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 08:53:49,096] Trial 84 finished with value: 0.07539483777557977 and parameters: {'LEARNING_RATE': 0.24983209861853112, 'USER_ALPHA': 0.004038671968281073, 'ITEM_ALPHA': 0.008495910066558357, 'N_EPOCHS': 49}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 09:02:55,375] Trial 85 finished with value: 0.0735826530283737 and parameters: {'LEARNING_RATE': 0.28656779293666146, 'USER_ALPHA': 0.0049267724028732385, 'ITEM_ALPHA': 0.008193964765549126, 'N_EPOCHS': 43}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 09:10:05,323] Trial 86 finished with value: 0.07597351326049384 and parameters: {'LEARNING_RATE': 0.37008073497708993, 'USER_ALPHA': 0.00440369402539047, 'ITEM_ALPHA': 0.007661184517989781, 'N_EPOCHS': 33}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 09:16:57,969] Trial 87 finished with value: 0.07328804826671531 and parameters: {'LEARNING_RATE': 0.36704100941901485, 'USER_ALPHA': 0.0042401296368574015, 'ITEM_ALPHA': 0.0075392196231400185, 'N_EPOCHS': 32}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 09:23:27,582] Trial 88 finished with value: 0.07348602430273686 and parameters: {'LEARNING_RATE': 0.3984222320926865, 'USER_ALPHA': 0.005215491409821222, 'ITEM_ALPHA': 0.00729318861916539, 'N_EPOCHS': 30}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 09:31:30,264] Trial 89 finished with value: 0.07523241843709605 and parameters: {'LEARNING_RATE': 0.3126786193541939, 'USER_ALPHA': 0.004461831484281197, 'ITEM_ALPHA': 0.006450740243415255, 'N_EPOCHS': 36}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 09:38:26,023] Trial 90 finished with value: 0.07433084653753672 and parameters: {'LEARNING_RATE': 0.3490547045162557, 'USER_ALPHA': 0.005951280350849641, 'ITEM_ALPHA': 0.00674344800381563, 'N_EPOCHS': 33}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 09:46:30,812] Trial 91 finished with value: 0.07536577628395023 and parameters: {'LEARNING_RATE': 0.38136750900201594, 'USER_ALPHA': 0.004696254063462046, 'ITEM_ALPHA': 0.00782263209174479, 'N_EPOCHS': 39}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 09:54:56,900] Trial 92 finished with value: 0.07558203287933991 and parameters: {'LEARNING_RATE': 0.2986439141451548, 'USER_ALPHA': 0.005507217512412387, 'ITEM_ALPHA': 0.007010647201010008, 'N_EPOCHS': 40}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 10:03:08,256] Trial 93 finished with value: 0.0749606621680243 and parameters: {'LEARNING_RATE': 0.2732789052470491, 'USER_ALPHA': 0.004848254854002855, 'ITEM_ALPHA': 0.008824895743611444, 'N_EPOCHS': 38}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 10:10:23,990] Trial 94 finished with value: 0.0741200170152297 and parameters: {'LEARNING_RATE': 0.33476714056913615, 'USER_ALPHA': 0.005132789022654675, 'ITEM_ALPHA': 0.007612636300890845, 'N_EPOCHS': 34}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 10:17:15,572] Trial 95 finished with value: 0.07292950990167814 and parameters: {'LEARNING_RATE': 0.3727831146567304, 'USER_ALPHA': 0.0038014275849648192, 'ITEM_ALPHA': 0.00817663191558608, 'N_EPOCHS': 28}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 10:26:13,678] Trial 96 finished with value: 0.07442466545181416 and parameters: {'LEARNING_RATE': 0.2511262716280167, 'USER_ALPHA': 0.005370827204512456, 'ITEM_ALPHA': 0.008544287834472933, 'N_EPOCHS': 42}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 10:36:23,655] Trial 97 finished with value: 0.07350874939776636 and parameters: {'LEARNING_RATE': 0.35144025164688714, 'USER_ALPHA': 0.005637833200846426, 'ITEM_ALPHA': 0.007362343451835673, 'N_EPOCHS': 50}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 10:46:10,180] Trial 98 finished with value: 0.07569759307603785 and parameters: {'LEARNING_RATE': 0.23008040511042452, 'USER_ALPHA': 0.004487635522017533, 'ITEM_ALPHA': 0.00902253243735608, 'N_EPOCHS': 44}. Best is trial 3 with value: 0.07675954767716515.
# [I 2023-12-04 10:56:00,096] Trial 99 finished with value: 0.07593830211370609 and parameters: {'LEARNING_RATE': 0.31521559580781994, 'USER_ALPHA': 0.004093714904796328, 'ITEM_ALPHA': 0.00803080123568635, 'N_EPOCHS': 47}. Best is trial 3 with value: 0.07675954767716515.


In [15]:
# Зададим моделям параметры, найденные при помощи optuna, для сравнения разных моделей
N_FACTORS = 32
models = {
    "popular": PopularModel(),
    # {'REGULARIZATION': 0.8739786271750147, 'ALPHA': 11, 'ITERATIONS': 9}
    f"ALS_{N_FACTORS}": ImplicitALSWrapperModel(
        model=AlternatingLeastSquares(
            factors=N_FACTORS,
            regularization=0.8739786271750147,
            alpha=11,
            iterations=9,
            random_state=RANDOM_STATE,
            num_threads=NUM_THREADS,
        ),
        fit_features_together=True,
    ),
    # {'LEARNING_RATE': 0.35440482510463406, 'USER_ALPHA': 0.005544087818166458, 'ITEM_ALPHA': 0.007499324044528365, 'N_EPOCHS': 36}
    f"LightFM_warp_{N_FACTORS}": LightFMWrapperModel(
        LightFM(
            no_components=N_FACTORS,
            loss="warp",
            random_state=RANDOM_STATE,
            learning_rate=0.35440482510463406,
            user_alpha=0.005544087818166458,
            item_alpha= 0.007499324044528365,
        ),
        epochs=36,
        num_threads=NUM_THREADS,
    ),
}



/root/RecSysITMO/.venv/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


In [16]:
metrics_name = {
    'Precision': Precision,
    'Recall': Recall,
    'MAP': MAP,
}

metrics = {}
for metric_name, metric in metrics_name.items():
    for k in range(1, 10+1):
        metrics[f'{metric_name}@{k}'] = metric(k=k)

In [17]:
models

{'popular': <rectools.models.popular.PopularModel at 0x7f96e2c564d0>,
 'ALS_32': <rectools.models.implicit_als.ImplicitALSWrapperModel at 0x7f96e6f527d0>,
 'LightFM_warp_32': <rectools.models.lightfm.LightFMWrapperModel at 0x7f96e23d4190>}

In [18]:
TEST_USERS = test[Columns.User].unique()

In [19]:
%%time
results = []
for model_name, model in models.items():
    print(f"Fitting model {model_name}...")
    model_quality = {'model': model_name}

    model.fit(dataset)
    recos = model.recommend(
        users=TEST_USERS,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )
    metric_values = calc_metrics(metrics, recos, test, train)
    model_quality.update(metric_values)
    results.append(model_quality)

Fitting model popular...
Fitting model ALS_32...


/root/RecSysITMO/.venv/lib/python3.10/site-packages/rectools/dataset/features.py:420: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")


Fitting model LightFM_warp_32...
CPU times: user 37min 3s, sys: 6min 46s, total: 43min 50s
Wall time: 14min 4s


In [20]:
df_quality = pd.DataFrame(results).T
df_quality.columns = df_quality.iloc[0]
df_quality.drop('model', inplace=True)

In [21]:
df_quality.style.highlight_max(color='green', axis=1)

model,popular,ALS_32,LightFM_warp_32
Precision@1,0.069368,0.082707,0.074378
Recall@1,0.035863,0.042494,0.039106
Precision@2,0.063681,0.068124,0.063593
Recall@2,0.064597,0.067975,0.064553
Precision@3,0.059233,0.059685,0.059207
Recall@3,0.088080,0.087639,0.088030
Precision@4,0.057348,0.053717,0.057144
Recall@4,0.112881,0.103802,0.112547
Precision@5,0.051035,0.048914,0.050669
Recall@5,0.124184,0.117067,0.123328


Судя по метрикам, для LightFM подобрались не лучшие гиперпараметры, т.к. даже со стандартными параметрами, метрики были выше

# Triain ALS

Обучим ALS отдельно для внедрения в систему

In [22]:
model = models["ALS_32"]

In [23]:
model.fit(dataset)
recos = model.recommend(
    users=TEST_USERS,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)

/root/RecSysITMO/.venv/lib/python3.10/site-packages/rectools/dataset/features.py:420: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")


## Save model and mapping

In [ ]:
# with open("../service/models/ALS_Online.pkl", "wb") as file: # Save ANN model
#     pickle.dump(model, file)

# with open("../service/models/ALS_Online.pkl", "rb") as file: 
#     model = pockle.load(file)

# with open("../service/models/als_user_id_map.pkl","wb") as file: # Save user_id_map
#     pickle.dump(dataset.user_id_map, file)
    
# with open("../service/models/als_item_id_map.pkl","wb") as file: # Save item_id_map
#     pickle.dump(dataset.item_id_map, file)

## Nmslib for ANN

In [24]:
import nmslib
import time

In [25]:
user_embeddings, item_embeddings = model.get_vectors()

In [26]:
K=10
space_name='negdotprod'
index = nmslib.init(method='hnsw', space=space_name, data_type=nmslib.DataType.DENSE_VECTOR) 
index.addDataPointBatch(item_embeddings) 

14019

In [27]:
M = 48
efC = 100

num_threads = 4
# Create an index
start = time.time()
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC}
index.createIndex(index_time_params) 
end = time.time() 
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

Index-time parameters {'M': 48, 'indexThreadQty': 4, 'efConstruction': 100}
Indexing time = 1.128071


In [28]:
efS = 100
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params)

Setting query-time parameters {'efSearch': 100}


In [29]:
query_matrix = user_embeddings[:1 :]
# Querying
query_qty = query_matrix.shape[0]
start = time.time() 
nbrs = index.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty))

kNN time total=0.003671 (sec), per query=0.003671 (sec), per query adjusted for thread number=0.014683 (sec)


In [30]:
nbrs

[(array([ 347,  112,   43,  749,   69, 1085,  335,  188,  279,  149],
        dtype=int32),
  array([-1.2314167 , -1.1247618 , -1.115778  , -1.1044744 , -1.0681179 ,
         -1.066022  , -1.0133785 , -1.0031286 , -0.99450815, -0.98760015],
        dtype=float32))]

In [31]:
# index.saveIndex(filename="../service/models/als_ann_index") # save ann index